In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as mcolors
from typing import Dict, Any
import names as n

from qdesignoptimizer.utils.plotting import load_data_by_date, plot_optimization_results
from qdesignoptimizer.sim_plot_progress import DataExtractor, OptimizationPlotter
from qdesignoptimizer.sim_plot_progress import OptPltSet
from qdesignoptimizer.utils.names_parameters import (
    param,
    param_capacitance,
    param_nonlin,
)

In [ ]:
import optimization_targets as ot

# Figures for publication

In [ ]:
# selected colors based on Gothenburg tram line map
colors_got = {'green': '#009639', 
              'blue': '#0066CC', 
              'yellow': '#FFD200', 
              'red': '#E30613', 
              'pink': '#EC008C', 
              'lightblue': '#00B5E2', 
              'purple': '#93328E', 
              'orange': '#F58220', 
              'grey': '#8C8C8C',
              'lightsalmon': '#FF9E80'}

In [ ]:
def lighten_color(hex_color, factor=0.3):
    """Lighten a hex color by mixing with white"""
    rgb = np.array(mcolors.hex2color(hex_color))
    # Mix with white: new_color = color + factor * (white - color)
    lightened = rgb + factor * (1 - rgb)
    return mcolors.rgb2hex(lightened)

def darken_color(hex_color, factor=0.3):
    """Darken a hex color by mixing with black"""
    rgb = np.array(mcolors.hex2color(hex_color))
    # Mix with black: new_color = color * (1 - factor)
    darkened = rgb * (1 - factor)
    return mcolors.rgb2hex(darkened)

def get_color_shades(hex_color, num_shades=5):
    """
    Generate a list of color shades from dark to light with original color in center.
    
    Parameters:
    -----------
    hex_color : str
        Base hex color (e.g., '#009639')
    num_shades : int
        Number of shades on each side of the original color
        Total colors returned = 2 * num_shades + 1
        
    Returns:
    --------
    list
        List of hex colors ordered from darkest to lightest
        [darkest, ..., darker, ORIGINAL, lighter, ..., lightest]
    """
    shades = []
    
    # Generate darker shades (from darkest to less dark)
    for i in range(num_shades, 0, -1):
        factor = i / (num_shades + 3)  # Avoid pure black by using num_shades+3
        dark_shade = darken_color(hex_color, factor)
        shades.append(dark_shade)
    
    # Add original color in the center
    shades.append(hex_color)
    
    # Generate lighter shades (from less light to lightest)
    for i in range(1, num_shades + 1):
        factor = i / (num_shades + 1)  # Avoid pure white by using num_shades+1
        light_shade = lighten_color(hex_color, factor)
        shades.append(light_shade)
    
    return shades

color_shades_purple = get_color_shades(colors_got['purple'], num_shades=5)
color_shades_red = get_color_shades(colors_got['red'], num_shades=5)
color_shades_blue = get_color_shades(colors_got['blue'], num_shades=5)
color_shades_lightblue = get_color_shades(colors_got['lightblue'], num_shades=5)
color_shades_grey = get_color_shades(colors_got['grey'], num_shades=5)

In [ ]:
def crop_image_by_fraction(image_path, left=0, right=0, top=0, bottom=0, rotate_90=0):
    """
    Crop an image by specified fractions from each edge and optionally rotate.
    
    Parameters:
    -----------
    image_path : str
        Path to the image file
    left : float
        Fraction to crop from left edge (0-1)
    right : float  
        Fraction to crop from right edge (0-1)
    top : float
        Fraction to crop from top edge (0-1)
    bottom : float
        Fraction to crop from bottom edge (0-1)
    rotate_90 : int
        Number of 90-degree rotations (0, 1, 2, 3 or negative values)
        0: no rotation, 1: 90° clockwise, 2: 180°, 3: 270° clockwise, etc.
        
    Returns:
    --------
    numpy.ndarray
        Cropped and rotated image array
    """
    # Read the image
    img = mpimg.imread(image_path)
    
    # Get dimensions
    height, width = img.shape[:2]  # Works for both grayscale and color images
    print(f"Original image size: {width}x{height}")
    
    # Calculate crop boundaries
    left_px = int(width * left)
    right_px = int(width * (1 - right))
    top_px = int(height * top)
    bottom_px = int(height * (1 - bottom))
    
    # Crop the image using array slicing
    # Format: img[top:bottom, left:right] for 2D
    # Format: img[top:bottom, left:right, :] for 3D (color)
    if len(img.shape) == 3:  # Color image
        cropped_img = img[top_px:bottom_px, left_px:right_px, :]
    else:  # Grayscale image
        cropped_img = img[top_px:bottom_px, left_px:right_px]
    
    new_height, new_width = cropped_img.shape[:2]
    print(f"Cropped image size: {new_width}x{new_height}")
    
    # Apply rotation if specified
    if rotate_90 != 0:
        # Normalize rotation to 0-3 range
        rotation_steps = rotate_90 % 4
        if rotation_steps != 0:
            cropped_img = np.rot90(cropped_img, k=rotation_steps)
            final_height, final_width = cropped_img.shape[:2]
            print(f"After {rotate_90 * 90}° rotation: {final_width}x{final_height}")
    
    return cropped_img

## Fig 2: Single qubit example

In [ ]:
parent_path = r"C:\Users\lukassp\OneDrive - Chalmers\202Q-lab\PROJECTS\QDesignOptimizer\Files_for_paper"
child_path = r"\single_qubit_resonator_fullchi"


### Load iamge

In [ ]:
cropped_image = crop_image_by_fraction(parent_path + child_path + r'\2025-08-10_OPT_qubit_resonator_final.png', left=0.1, right=0.1, top=0.1, bottom=0.1, rotate_90=1)


### Load data

In [ ]:
f_sim_result = [parent_path + child_path + r"\multi_transmon_chip_20250810-104840.npy"]

In [ ]:
d_sim_result = np.load(f_sim_result[0], allow_pickle=True)[0]
system_target_params = d_sim_result["system_target_params"]
opt_result = d_sim_result['optimization_results']

opt_target_list = ot.get_opt_targets_2qubits_resonator_coupler([1], True,True,True,True,True,False,False)

iteration = 'Iteration'
config = {
    'RES_freq': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.FREQ), y_label="RES Freq", unit="GHz"), 
    'RES_kappa': OptPltSet(n.ITERATION, param(n.RESONATOR_1, n.KAPPA), y_label="RES Kappa", unit="MHz"),
    'QUB_freq': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="QB Freq", unit="GHz"),
    'QUB_anh': OptPltSet(n.ITERATION, param_nonlin(n.QUBIT_1, n.QUBIT_1), y_label="QB Anharm.", unit="MHz"),
    'RES_QUB_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_1, n.QUBIT_1), y_label="RES-QB Chi", unit="MHz"),
    }

print(system_target_params)
print(opt_result[0]['design_variables'])


In [ ]:
data_extractor = DataExtractor([opt_result], system_target_params, opt_target_list)
plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)

### Plot figure with data

In [ ]:
figure_width_inch = 7
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, 3.5))
gs = fig.add_gridspec(2, 4, wspace=0.7, hspace=0.4, left=0.02, right=0.96, bottom=0.13, top=0.95)
ax0 = fig.add_subplot(gs[0:2, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 2])
ax5 = fig.add_subplot(gs[0, 3])
ax6 = fig.add_subplot(gs[1, 3])

plotter._plot_single_param(ax1, config['RES_freq'], y_param=config['RES_freq'].y, color=colors_got['blue'])
plotter._plot_single_param(ax2, config['RES_kappa'], y_param=config['RES_kappa'].y, color=colors_got['lightblue'])
plotter._plot_single_param(ax3, config['QUB_freq'], y_param=config['QUB_freq'].y, color=colors_got['purple'])
plotter._plot_single_param(ax4, config['QUB_anh'], y_param=config['QUB_anh'].y, color=colors_got['pink'])
plotter._plot_single_param(ax5, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['orange'])
plotter._plot_param_vs_design_var(ax6, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['orange'], sort_by_x=False)

ax1.set_xticks([5,10,15])
ax2.set_xticks([5,10,15])
ax3.set_xticks([5,10,15])
ax4.set_xticks([5,10,15])
ax5.set_xticks([5,10,15])
ax6.set_xticks([100,300])

ax1.set_xlabel('Iteration')
ax2.set_xlabel('Iteration')
ax3.set_xlabel('Iteration')
ax4.set_xlabel('Iteration')
ax5.set_xlabel('Iteration')
ax6.set_xlabel('Coupling length (um)')

ax1.text(0.1,0.88,'(b)', transform=ax1.transAxes)
ax2.text(0.04,0.88,'(c)', transform=ax2.transAxes)
ax3.text(0.1,0.88,'(d)', transform=ax3.transAxes)
ax4.text(0.1,0.88,'(e)', transform=ax4.transAxes)
ax5.text(0.1,0.88,'(f)', transform=ax5.transAxes)
ax6.text(0.16,0.88,'(g)', transform=ax6.transAxes)

ax0.imshow(cropped_image)
ax0.axis('off')

fig.savefig(parent_path + r"\Fig2_OPT_single_qubit_resonator.png", dpi=300)

## Fig 3: Effect of nonlinear model

In [ ]:
iteration = 'Iteration'
config = {
    'RES_QUB_chi': OptPltSet(n.ITERATION, param_nonlin(n.RESONATOR_1, n.QUBIT_1), y_label="RES-QB Chi", unit="MHz"),
    'QUB_freq': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="QB Freq", unit="GHz")
    }

### Load data (nonlinear model)

In [ ]:
child_path_simple_chi = r"\single_qubit_resonator_simple"

In [ ]:
f_sim_result = [parent_path + child_path + r"\multi_transmon_chip_20250810-104840.npy", parent_path + child_path_simple_chi + r"\multi_transmon_chip_20250810-093400.npy"] 

In [ ]:
opt_plot = []
for i in range(len(f_sim_result)):
    d_sim_result = np.load(f_sim_result[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot.append(plotter)

### Load data (variations)

In [ ]:
f_sim_result_variation = [parent_path + child_path + r"\multi_transmon_chip_20250810-104840.npy", parent_path + child_path_simple_chi + r"\multi_transmon_chip_20250810-093400.npy"] 

In [ ]:
opt_plot_variations = []
for i in range(len(f_sim_result_variation)):
    d_sim_result = np.load(f_sim_result_variation[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot_variations.append(plotter)

### Plot figure with data

In [ ]:
figure_width_inch = 3.35
figure_height_inch = 3.5
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, figure_height_inch))
gs = fig.add_gridspec(2, 2, wspace=0.7, hspace=0.4, left=0.16, right=0.96, bottom=0.14, top=0.95)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

opt_plot[0]._plot_single_param(ax1, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['blue'])
opt_plot[1]._plot_single_param(ax1, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=colors_got['lightblue'])
opt_plot[1]._plot_target(ax1, config['RES_QUB_chi'].y, 0, 15, colors_got['grey'], config['RES_QUB_chi'].normalization)

ax1.set_xticks([5,10])
ax1.set_xlabel('Iteration')

n_variations = len(opt_plot_variations)
for i in range(n_variations):
    opt_plot_variations[i]._plot_single_param(ax2, config['QUB_freq'], y_param=config['QUB_freq'].y, color=color_shades_purple[i])
    opt_plot_variations[i]._plot_single_param(ax3, config['RES_QUB_chi'], y_param=config['RES_QUB_chi'].y, color=color_shades_blue[i])


ax1.text(0.1,0.88,'(a)', transform=ax1.transAxes)
ax2.text(0.1,0.88,'(b)', transform=ax2.transAxes)
ax3.text(0.1,0.88,'(c)', transform=ax3.transAxes)

ax2.set_xticks([5,10])
ax3.set_xticks([5,10])
ax2.set_xlabel('Iteration')
ax3.set_xlabel('Iteration')


fig.savefig(parent_path + r"\Fig3_COMP_single_qubit_resonator.png", dpi=300)

## Fig 4: Capacitance targets

### Load images

In [ ]:
child_path_capacitance = r'\capacitance_resonator_feedline'
child_path_decay = r"\chargeline_qubit"

In [ ]:
cropped_image_capacitance = crop_image_by_fraction(parent_path + child_path_capacitance + r'\2025-08-10_OPT_resonator_feedline_capacitance_final_zoom.png', left=0.1, right=0.1, top=0.0, bottom=0.1, rotate_90=0)
cropped_image_decay = crop_image_by_fraction(parent_path + child_path_decay + r'\2025-08-10_OPT_chargeline_final_zoom.png', left=0., right=0.0, top=0.0, bottom=0.1, rotate_90=0)

### Load data

In [ ]:
f_sim_result = [parent_path  + child_path_capacitance + r'\multi_transmon_chip_20250810-122434.npy', parent_path + child_path_decay + r'\multi_transmon_chip_20250810-134018.npy']

In [ ]:
opt_plot = []
for i in range(len(f_sim_result)):
    d_sim_result = np.load(f_sim_result[i], allow_pickle=True)[0]
    system_target_params = d_sim_result["system_target_params"]
    opt_result = d_sim_result['optimization_results']

    data_extractor = DataExtractor([opt_result], system_target_params)
    plotter = OptimizationPlotter(data_extractor, plot_variance=False, save_figures=False)
    opt_plot.append(plotter)

In [ ]:
iteration = 'Iteration'
config = {
    'res_fd_coupling': OptPltSet(n.ITERATION, param_capacitance("prime_cpw_name_tee1_", "second_cpw_name_tee1_"), y_label="Capacitance", unit="fF"), 
    'charge_line': OptPltSet(n.ITERATION, param(n.QUBIT_1, n.CHARGE_LINE_LIMITED_T1), y_label="T1 limit", y_scale="linear", unit="ms")
    }

### Plot figure with data and images

In [ ]:
figure_width_inch = 3.35
figure_height_inch = 3.2
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'], 
    'font.size': 10,  
    'mathtext.fontset': 'stix',  
})

fig = plt.figure(figsize=(figure_width_inch, figure_height_inch))
gs = fig.add_gridspec(2, 2, wspace=0.7, hspace=0.4, left=0.16, right=0.96, bottom=0.14, top=0.95)
ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[1,0])
ax3 = fig.add_subplot(gs[0,1])
ax4 = fig.add_subplot(gs[1,1])

ax1.imshow(cropped_image_capacitance)
ax2.imshow(cropped_image_decay)

ax1.axis('off')
ax2.axis('off')


opt_plot[0]._plot_single_param(ax3, config['res_fd_coupling'], y_param=config['res_fd_coupling'].y, color=colors_got['blue'])
opt_plot[1]._plot_single_param(ax4, config['charge_line'], y_param=config['charge_line'].y, color=colors_got['lightblue'])

ax3.set_xticks([5,10,15])
ax4.set_xticks([5,10,15])

ax3.set_xlabel('Iteration')
ax4.set_xlabel('Iteration')

ax3.text(0.1,0.88,'(c)', transform=ax3.transAxes)
ax4.text(0.1,0.88,'(d)', transform=ax4.transAxes)

fig.savefig(parent_path + r"\Fig4_OPT_capacitance_decay.png", dpi=300)

In [ ]:
# Load data

opt_result = d_sim_result['optimization_results']
system_target_params = d_sim_result["system_target_params"]

In [ ]:
system_target_params

In [ ]:
PLOT_SETTINGS = {
    "RES": [
        OptPltSet(
            n.ITERATION, param(n.RESONATOR_1, n.FREQ), y_label="RES Freq", unit="GHz"
        ),
        OptPltSet(
            n.ITERATION, param(n.RESONATOR_1, n.KAPPA), y_label="RES Kappa", unit="MHz"
        ),
        OptPltSet(
            n.design_var_length(n.RESONATOR_1),
            param(n.RESONATOR_1, n.KAPPA),
            y_label="RES Kappa",
            unit="MHz",
        ),  # As an example that design variables can also be plotted for the results
    ],
    "QUBIT": [
        OptPltSet(n.ITERATION, param(n.QUBIT_1, n.FREQ), y_label="QB Freq", unit="GHz"),
        OptPltSet(
            n.ITERATION,
            param_nonlin(n.QUBIT_1, n.QUBIT_1),
            y_label="QB Anharm.",
            unit="MHz",
        ),
    ],
    "COUPLINGS": [
        OptPltSet(
            n.ITERATION,
            param_nonlin(n.RESONATOR_1, n.QUBIT_1),
            y_label="RES-QB Chi",
            unit="kHz",
        ),
    ],
}

In [ ]:
data_extractor = DataExtractor(opt_result, system_target_params)
plotter = OptimizationPlotter(data_extractor)

In [ ]:
config= PLOT_SETTINGS['QUBIT'][0]

In [ ]:
config.y

In [ ]:
data_extractor.get_parameter_value?

In [ ]:
data_extractor.get_parameter_value(config.y, opt_result,0)

In [ ]:
data_extractor.extract_xy_data(config.x, config.y, run_index=2)

In [ ]:
fig, axs = plt.subplots(2,3)
# plotter._setup_ax(axs[1,1],PLOT_SETTINGS['QUBIT'][0],x_label='Iteration', y_label='Qubit freq')
plotter._plot_single_param(axs[1,1], PLOT_SETTINGS['QUBIT'][0], y_param=PLOT_SETTINGS['QUBIT'][0].y, color='blue')

In [ ]:
OptimizationResult = Dict[str, Any]
data_extractor.get_parameter_value('freq_qubit_1', OptimizationResult, iteration=)

In [ ]:
data_extractor.get_parameter_value(param_name='qubit_1_freq', re)

In [ ]:
opt_result

In [ ]:
d_sim_result

In [ ]:
plot_optimization_results(f_sim_result)